In [8]:
import xarray as xr
import pandas as pd
import geopandas as gpd
from tqdm import tqdm  # Optional: For progress tracking
from scipy.interpolate import griddata
from statsmodels.tsa.seasonal import seasonal_decompose


## Cleaning

In [9]:

import unicodedata

# Function to normalize the names
def normalize_municipality_name(name):
    if type(name) == float:
        return name
    else:
        # Normalize the string (remove diacritical marks)
        name_without_diacritics = unicodedata.normalize('NFKD', name).encode('ascii', 'ignore').decode('ascii')
        # Convert to lowercase
        return name_without_diacritics.lower()



In [10]:
mun_gdf = gpd.read_file(r'../../../../../portugal/municipality_data/municipalities-shapefile-2/concelhos.shp')
mun_metadata = pd.read_excel(r'../../../../../portugal/municipality_data/concelhos-metadata.xlsx', dtype={'dicofre' : 'string'})

mun_metadata = mun_metadata[['dicofre','designacao']]

mun_gdf = mun_gdf.merge(mun_metadata, left_on='CCA_2', right_on='dicofre').drop(columns='NAME_2')
mun_gdf = mun_gdf.rename(columns={'designacao': 'NAME_2'})
mun_gdf['concelho'] = mun_gdf['NAME_2'].map(normalize_municipality_name)
mun_gdf = mun_gdf[~mun_gdf['NAME_1'].isin(['Azores', 'Madeira'])]
west, south, east, north = mun_gdf.total_bounds

In [11]:
PORTUGAL_BBOX = {
    "min_lat": south,  # Minimum latitude
    "max_lat": north,  # Maximum latitude
    "min_lon": west,   # Minimum longitude
    "max_lon": east    # Maximum longitude
}


In [12]:
PORTUGAL_BBOX

{'min_lat': 36.960971832275334,
 'max_lat': 42.15431594848633,
 'min_lon': -9.547081947326603,
 'max_lon': -6.189142227172852}

In [13]:
def kelvin_to_celsius(kelvin_values):
    """
    Convert temperature values from Kelvin to Celsius.

    Parameters:
    - kelvin_values (numpy array or list): Temperature values in Kelvin.

    Returns:
    - numpy array: Temperature values in Celsius.
    """
    celsius_values = kelvin_values - 273.15
    return celsius_values

get time from file names

### ECDE indicators catalogue entry (Climate indicators for Europe from 1940 to 2100 derived from reanalysis and climate projections)

#### Tropical Nights

In [17]:
tropical_df = xr.open_dataset(r'/Volumes/Dhruv_External_Disk/climate_data/climate_indic_temp_prec/05_tropical_nights-reanalysis-monthly-grid-1940-2023-v1.0.nc').to_dataframe().reset_index()
tropical_df = tropical_df[
    (tropical_df["lat"] >= PORTUGAL_BBOX["min_lat"]) &
    (tropical_df["lat"] <= PORTUGAL_BBOX["max_lat"]) &
    (tropical_df["lon"] >= PORTUGAL_BBOX["min_lon"]) &
    (tropical_df["lon"] <= PORTUGAL_BBOX["max_lon"])
]

In [ ]:
tropical_df.rename(columns={'t2m': 'num_tropical_nights'}, inplace=True)
tropical_df.drop(columns=['realization'], inplace=True)

In [23]:
tropical_df

,time,lat,lon,num_tropical_nights
11432,1940-01-01,37.0,-9.50,0
11433,1940-01-01,37.0,-9.25,0
11434,1940-01-01,37.0,-9.00,0
11435,1940-01-01,37.0,-8.75,0
11436,1940-01-01,37.0,-8.50,0
...,...,...,...,...
50502806,2023-12-01,42.0,-7.25,0
50502807,2023-12-01,42.0,-7.00,0
50502808,2023-12-01,42.0,-6.75,0
50502809,2023-12-01,42.0,-6.50,0


In [25]:
tropical_df.to_parquet("/Volumes/Dhruv_External_Disk/climate_data/temperature/tropical_nights_portugal_1940_2023.parquet", engine="pyarrow", compression="snappy")

#### Heat Waves

In [26]:
heatwaves_df = xr.open_dataset(r'/Volumes/Dhruv_External_Disk/climate_data/climate_indic_temp_prec/09_heat_waves_climatological-reanalysis-monthly-grid-1940-2023-v1.0.nc').to_dataframe().reset_index()
heatwaves_df = heatwaves_df[
    (heatwaves_df["lat"] >= PORTUGAL_BBOX["min_lat"]) &
    (heatwaves_df["lat"] <= PORTUGAL_BBOX["max_lat"]) &
    (heatwaves_df["lon"] >= PORTUGAL_BBOX["min_lon"]) &
    (heatwaves_df["lon"] <= PORTUGAL_BBOX["max_lon"])
]


In [29]:
heatwaves_df.rename(columns={'data': 'num_heatwaves'}, inplace=True)
heatwaves_df.drop(columns=['realization'], inplace=True)


In [31]:
heatwaves_df.to_parquet("/Volumes/Dhruv_External_Disk/climate_data/temperature/heatwaves_portugal_1940_2023.parquet", engine="pyarrow", compression="snappy")

#### Frost Days

In [5]:
frost_df = xr.open_dataset(r'/Volumes/Dhruv_External_Disk/climate_data/climate_indic_temp_prec/11_frost_days-reanalysis-monthly-grid-1940-2023-v1.0.nc').to_dataframe().reset_index()
frost_df = frost_df[
    (frost_df["lat"] >= PORTUGAL_BBOX["min_lat"]) &
    (frost_df["lat"] <= PORTUGAL_BBOX["max_lat"]) &
    (frost_df["lon"] >= PORTUGAL_BBOX["min_lon"]) &
    (frost_df["lon"] <= PORTUGAL_BBOX["max_lon"])
]

In [8]:
frost_df.drop(columns=['realization'], inplace=True)
frost_df.rename(columns={'t2m' : 'frost_days'}, inplace=True)


In [9]:
frost_df.to_parquet("/Volumes/Dhruv_External_Disk/climate_data/temperature/frost_portugal_1940_2023.parquet", engine="pyarrow", compression="snappy")

#### High UTCI Days

In [32]:
utci_df = xr.open_dataset(r'/Volumes/Dhruv_External_Disk/climate_data/climate_indic_temp_prec/10_high_utci_days-reanalysis-monthly-grid-1940-2023-v1.0.nc').to_dataframe().reset_index()
utci_df = utci_df[
    (utci_df["lat"] >= PORTUGAL_BBOX["min_lat"]) &
    (utci_df["lat"] <= PORTUGAL_BBOX["max_lat"]) &
    (utci_df["lon"] >= PORTUGAL_BBOX["min_lon"]) &
    (utci_df["lon"] <= PORTUGAL_BBOX["max_lon"])
]

In [34]:
utci_df.drop(columns=['height'], inplace=True)


In [35]:
utci_df.to_parquet("/Volumes/Dhruv_External_Disk/climate_data/temperature/utci_portugal_1940_2023.parquet", engine="pyarrow", compression="snappy")

#### Consecutive Dry Days

In [36]:
dry_days_df = xr.open_dataset(r'/Volumes/Dhruv_External_Disk/climate_data/climate_indic_temp_prec/18_consecutive_dry_days-reanalysis-monthly-grid-1940-2023-v1.0.nc').to_dataframe().reset_index()
dry_days_df = dry_days_df[
    (dry_days_df["lat"] >= PORTUGAL_BBOX["min_lat"]) &
    (dry_days_df["lat"] <= PORTUGAL_BBOX["max_lat"]) &
    (dry_days_df["lon"] >= PORTUGAL_BBOX["min_lon"]) &
    (dry_days_df["lon"] <= PORTUGAL_BBOX["max_lon"])
]

In [38]:
dry_days_df.drop(columns=['realization'], inplace=True)

In [39]:
dry_days_df.to_parquet("/Volumes/Dhruv_External_Disk/climate_data/precipitation/dry_days_portugal_1940_2023.parquet", engine="pyarrow", compression="snappy")

#### Fire

##### High Fire Danger

In [40]:
high_fire_df = xr.open_dataset(r'/Volumes/Dhruv_External_Disk/climate_data/climate_indic_temp_prec/22_days_with_high_fire_danger-reanalysis-monthly-grid-1979-2021-v1.0.nc').to_dataframe().reset_index()   
high_fire_df = high_fire_df[
    (high_fire_df["lat"] >= PORTUGAL_BBOX["min_lat"]) &
    (high_fire_df["lat"] <= PORTUGAL_BBOX["max_lat"]) &
    (high_fire_df["lon"] >= PORTUGAL_BBOX["min_lon"]) &
    (high_fire_df["lon"] <= PORTUGAL_BBOX["max_lon"])
]   

In [44]:
high_fire_df.fillna(0, inplace=True)
high_fire_df.to_parquet("/Volumes/Dhruv_External_Disk/climate_data/fire/high_fire_days_portugal_1979_2021.parquet", engine="pyarrow", compression="snappy")

##### Fire Weather Index

In [47]:
fwi_df =  xr.open_dataset(r'/Volumes/Dhruv_External_Disk/climate_data/climate_indic_temp_prec/l6_fire_weather_index-reanalysis-monthly-grid-1979-2021-v1.0.nc').to_dataframe().reset_index()

fwi_df = fwi_df[
    (fwi_df["lat"] >= PORTUGAL_BBOX["min_lat"]) &
    (fwi_df["lat"] <= PORTUGAL_BBOX["max_lat"]) &
    (fwi_df["lon"] >= PORTUGAL_BBOX["min_lon"]) &
    (fwi_df["lon"] <= PORTUGAL_BBOX["max_lon"])
]   


In [51]:
fwi_df.fillna(0, inplace=True)
fwi_df.to_parquet("/Volumes/Dhruv_External_Disk/climate_data/fire/fire_weather_index_portugal_1979_2021.parquet", engine="pyarrow", compression="snappy")

#### Extreme Precipitation

In [52]:
extreme_prep = xr.open_dataset(r'/Volumes/Dhruv_External_Disk/climate_data/climate_indic_temp_prec/14_extreme_precipitation_total-reanalysis-monthly-grid-1940-2023-v1.0.nc').to_dataframe().reset_index()
extreme_prep = extreme_prep[
    (extreme_prep["lat"] >= PORTUGAL_BBOX["min_lat"]) &
    (extreme_prep["lat"] <= PORTUGAL_BBOX["max_lat"]) &
    (extreme_prep["lon"] >= PORTUGAL_BBOX["min_lon"]) &
    (extreme_prep["lon"] <= PORTUGAL_BBOX["max_lon"])
]

In [54]:
extreme_prep.rename(columns={'data': 'extreme_precipitation_mm'}, inplace=True)
extreme_prep.drop(columns=['realization'], inplace=True)

In [55]:
extreme_prep.to_parquet("/Volumes/Dhruv_External_Disk/climate_data/precipitation/extreme_precipitation_portugal_1940_2023.parquet", engine="pyarrow", compression="snappy")

#### Extreme Precipitation Frequency

In [57]:
freq_ex_prep = xr.open_dataset(r'/Volumes/Dhruv_External_Disk/climate_data/climate_indic_temp_prec/15_frequency_of_extreme_precipitation-reanalysis-monthly-grid-1940-2023-v1.0.nc').to_dataframe().reset_index()
freq_ex_prep = freq_ex_prep[
    (freq_ex_prep["lat"] >= PORTUGAL_BBOX["min_lat"]) &
    (freq_ex_prep["lat"] <= PORTUGAL_BBOX["max_lat"]) &
    (freq_ex_prep["lon"] >= PORTUGAL_BBOX["min_lon"]) &
    (freq_ex_prep["lon"] <= PORTUGAL_BBOX["max_lon"])
]

In [59]:
freq_ex_prep.rename(columns={'data': 'frequency_of_extreme_precipitation'}, inplace=True)
freq_ex_prep.drop(columns=['realization'], inplace=True)

In [60]:
freq_ex_prep.to_parquet("/Volumes/Dhruv_External_Disk/climate_data/precipitation/frequency_of_extreme_precipitation_portugal_1940_2023.parquet", engine="pyarrow", compression="snappy")

#### Extreme Wind Speed

In [61]:
ex_wind = xr.open_dataset(r'/Volumes/Dhruv_External_Disk/climate_data/climate_indic_temp_prec/24_extreme_wind_speed_days-reanalysis-monthly-grid-1940-2023-v1.0.nc').to_dataframe().reset_index()  
ex_wind = ex_wind[
    (ex_wind["lat"] >= PORTUGAL_BBOX["min_lat"]) &
    (ex_wind["lat"] <= PORTUGAL_BBOX["max_lat"]) &
    (ex_wind["lon"] >= PORTUGAL_BBOX["min_lon"]) &
    (ex_wind["lon"] <= PORTUGAL_BBOX["max_lon"])
]


In [63]:
ex_wind.rename(columns={'data': 'extreme_wind_speed_days'}, inplace=True)
ex_wind.drop(columns=['realization'], inplace=True)


In [64]:
ex_wind.to_parquet("/Volumes/Dhruv_External_Disk/climate_data/wind/extreme_wind_speed_days_portugal_1940_2023.parquet", engine="pyarrow", compression="snappy")

#### Max 5 Day Precipitation

In [97]:
max_5_prep = xr.open_dataset(r'/Volumes/Dhruv_External_Disk/climate_data/precipitation/13_maximum_five_days_precipitation-reanalysis-monthly-grid-1940-2023-v1.0.nc').to_dataframe().reset_index()
max_5_prep = max_5_prep[
    (max_5_prep["lat"] >= PORTUGAL_BBOX["min_lat"]) &
    (max_5_prep["lat"] <= PORTUGAL_BBOX["max_lat"]) &
    (max_5_prep["lon"] >= PORTUGAL_BBOX["min_lon"]) &
    (max_5_prep["lon"] <= PORTUGAL_BBOX["max_lon"])
]

In [99]:
max_5_prep.rename(columns={'tp': 'maximum_five_days_precipitation_mm'}, inplace=True)
max_5_prep.drop(columns=['realization'], inplace=True)

In [100]:
max_5_prep.to_parquet("/Volumes/Dhruv_External_Disk/climate_data/precipitation/maximum_five_days_precipitation_portugal_1940_2023.parquet", engine="pyarrow", compression="snappy")

#### Hot Days

In [117]:
hot_days = xr.open_dataset(r'/Volumes/Dhruv_External_Disk/climate_data/temperature/06_hot_days-reanalysis-monthly-30deg-grid-1940-2023-v1.0.nc').to_dataframe().reset_index()
hot_days = hot_days[
    (hot_days["lat"] >= PORTUGAL_BBOX["min_lat"]) &
    (hot_days["lat"] <= PORTUGAL_BBOX["max_lat"]) &
    (hot_days["lon"] >= PORTUGAL_BBOX["min_lon"]) &
    (hot_days["lon"] <= PORTUGAL_BBOX["max_lon"])
]


In [119]:
hot_days.rename(columns={'t2m': 'hot_days'}, inplace=True)
hot_days.drop(columns=['realization'], inplace=True)

In [120]:
hot_days.to_parquet("/Volumes/Dhruv_External_Disk/climate_data/temperature/hot_days_portugal_1940_2023.parquet", engine="pyarrow", compression="snappy")

### ERA5

#### Vegetation

In [ ]:
nc_1 = xr.open_dataset(r'/Volumes/Dhruv_External_Disk/climate_data/vegetation/data_0.nc').to_dataframe().reset_index()
nc_2 = xr.open_dataset(r'/Volumes/Dhruv_External_Disk/climate_data/vegetation/data_1.nc').to_dataframe().reset_index()
veg_df = pd.concat([nc_1, nc_2])
veg_df.drop(columns=['expver', 'number', 't2m', 'tp'], inplace=True)
veg_df = veg_df.rename(columns={'valid_time': 'time'})



,time,latitude,longitude,cvl,cvh,lai_lv,lai_hv
0,2000-01-01 00:00:00,43.0,-32.00,0.0,0.0,0.0,0.0
1,2000-01-01 00:00:00,43.0,-31.75,0.0,0.0,0.0,0.0
2,2000-01-01 00:00:00,43.0,-31.50,0.0,0.0,0.0,0.0
3,2000-01-01 00:00:00,43.0,-31.25,0.0,0.0,0.0,0.0
4,2000-01-01 00:00:00,43.0,-31.00,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
1675060,2025-01-01 06:00:00,30.0,-7.00,NaN,NaN,NaN,NaN
1675061,2025-01-01 06:00:00,30.0,-6.75,NaN,NaN,NaN,NaN
1675062,2025-01-01 06:00:00,30.0,-6.50,NaN,NaN,NaN,NaN
1675063,2025-01-01 06:00:00,30.0,-6.25,NaN,NaN,NaN,NaN


In [32]:
veg_df.to_parquet(r'/Volumes/Dhruv_External_Disk/climate_data/vegetation/vegetation_m_PT_2000_2024.parquet')

#### Soil

In [116]:
soil_df = xr.open_dataset(r'/Volumes/Dhruv_External_Disk/climate_data/topography/soil_type.nc').to_dataframe().reset_index()
soil_df.drop(columns=['number', 'expver'], inplace=True) 
soil_df.rename(columns={'slt': 'soil_type'}, inplace=True)
soil_df.to_parquet("/Volumes/Dhruv_External_Disk/climate_data/topography/soil_type_portugal.parquet", engine="pyarrow", compression="snappy")

#### Riverine

In [53]:
import os

# Define the folder containing the NetCDF files
nc_folder = "/Volumes/Dhruv_External_Disk/climate_data/river_discharge"

# Initialize lists to store mean and standard deviation DataFrames
dfs_mean = []
dfs_std = []

# List all NetCDF files in the folder
nc_files = [file for file in os.listdir(nc_folder) if file.endswith(".nc")]

# Iterate through all NetCDF files with a tqdm progress bar
for file in tqdm(nc_files, desc="Processing NetCDF Files", unit="file"):
    file_path = os.path.join(nc_folder, file)

    # Open the NetCDF file with xarray
    ds = xr.open_dataset(file_path)

    # Resample to monthly frequency using mean and convert to DataFrame
    df_mean = (ds.resample(valid_time='M')
                    .mean('valid_time')
                    .to_dataframe()
                    .reset_index())

    # Resample to monthly frequency using standard deviation and convert to DataFrame
    df_std = (ds.resample(valid_time='M')
                    .std('valid_time')
                    .to_dataframe()
                    .reset_index())

    # Adjust longitude values
    df_mean['longitude'] = df_mean['longitude'] - 360
    df_std['longitude'] = df_std['longitude'] - 360

    # Drop the 'surface' column if it exists
    df_mean.drop(columns=['surface'], errors='ignore', inplace=True)
    df_std.drop(columns=['surface'], errors='ignore', inplace=True)

    # Rename the 'dis24' column appropriately
    df_mean.rename(columns={'dis24': 'river_discharge_mean', 'valid_time' : 'time'}, inplace=True)
    df_std.rename(columns={'dis24': 'river_discharge_std', 'valid_time' : 'time'}, inplace=True)

    # Append the processed DataFrames to their respective lists
    dfs_mean.append(df_mean)
    dfs_std.append(df_std)

# Concatenate all processed DataFrames into single DataFrames
river_mean_df = pd.concat(dfs_mean, ignore_index=True)
river_std_df = pd.concat(dfs_std, ignore_index=True)



Processing NetCDF Files:   0%|          | 0/12 [00:00<?, ?file/s]/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Library/Frameworks/Python.framewor

In [54]:
river_mean_df.fillna(0, inplace=True)
river_std_df.fillna(0, inplace=True)


In [55]:
river_mean_df.to_parquet(r'/Volumes/Dhruv_External_Disk/climate_data/river_discharge/river_dis_mean_2010_2022_PT_m.parquet')
river_std_df.to_parquet(r'/Volumes/Dhruv_External_Disk/climate_data/river_discharge/river_dis_std_2010_2022_PT_m.parquet')

#### Hourly Data

In [26]:
reanalysis_nc = xr.open_dataset(r'/Volumes/Dhruv_External_Disk/climate_data/reanalysis_selected.nc')
reanalysis_df = reanalysis_nc.to_dataframe()

In [15]:
reanalysis_nc

<xarray.Dataset> Size: 28MB
Dimensions:    (longitude: 14, latitude: 21, expver: 2, time: 1006)
Coordinates:
  * longitude  (longitude) float32 56B -9.5 -9.25 -9.0 ... -6.75 -6.5 -6.25
  * latitude   (latitude) float32 84B 41.92 41.67 41.42 ... 37.42 37.17 36.92
  * expver     (expver) int32 8B 1 5
  * time       (time) datetime64[ns] 8kB 1940-01-01 1940-02-01 ... 2023-10-01
Data variables:
    t2m        (time, expver, latitude, longitude) float64 5MB 284.7 ... 295.2
    msl        (time, expver, latitude, longitude) float64 5MB 1.014e+05 ... ...
    sst        (time, expver, latitude, longitude) float64 5MB 286.4 ... nan
    stl2       (time, expver, latitude, longitude) float64 5MB 286.4 ... 297.3
    sp         (time, expver, latitude, longitude) float64 5MB 1.014e+05 ... ...
    tp         (time, expver, latitude, longitude) float64 5MB 0.004636 ... 0...
Attributes:
    Conventions:  CF-1.6
    history:      2023-12-01 20:45:04 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [19]:
import os
# Path to the folder containing the NetCDF files
folder_path = '/Volumes/Dhruv_External_Disk/climate_data/net_cdf_hourly'

# List to store individual DataFrames
monthly_dfs = []

# Loop through each file in the folder
for file in os.listdir(folder_path):
    if file.endswith('.nc'):  # Check if the file is a NetCDF file
        file_path = os.path.join(folder_path, file)
        
        # Open the NetCDF file as an xarray dataset
        hourly_ds = xr.open_dataset(file_path)
        # Drop Irrelevant Variables 
        hourly_ds = hourly_ds.drop_vars(['stl2', 'sst', 'msl', 'sp'])
        hourly_ds['tp'] = hourly_ds['tp'] * 1000
        # Resample Data On Day, Aggregating 
        daily_means = hourly_ds.resample(time='D').mean('time')
        # Resample to daily totals by summing the hourly precipitation for each day
        daily_totals = hourly_ds.resample(time='D').sum('time')
        daily_means['tp_mm_day'] = daily_totals['tp']
        # Resample Data On Month, Aggregating
        monthly_stddevs = daily_means.resample(time='ME').std('time')
        
        # Convert the xarray dataset to a DataFrame
        monthly_df = monthly_stddevs.to_dataframe()

        # Append the DataFrame to the list
        monthly_dfs.append(monthly_df)

# Concatenate all individual DataFrames into one
erA5_std_df = pd.concat(monthly_dfs, axis=0).reset_index()



## Obtaining historical mean values, and deviations from historical means

In [27]:
reanalysis_df['tp'] = reanalysis_df['tp'] * 1000
mean_monthly_values = reanalysis_df.loc[pd.IndexSlice[:, :, :, '1940-01-01':'1980-12-01']].groupby(['longitude','latitude']).mean()
historical_deviations = reanalysis_df - mean_monthly_values


In [29]:
historical_deviations = historical_deviations.reset_index()
historical_deviations = historical_deviations[['time', 'latitude', 'longitude', 't2m', 'tp']].copy()
historical_deviations = historical_deviations[
    (historical_deviations.time >= '2000-01-01') & (historical_deviations.time <= '2022-12-31')
]


In [31]:
erA5_std_df

,time,latitude,longitude,t2m,tp,tp_mm_day
0,2015-01-31,41.919998,-9.50,0.874659,0.222424,5.338166
1,2015-01-31,41.919998,-9.25,0.912356,0.266698,6.400749
2,2015-01-31,41.919998,-9.00,0.995148,0.359232,8.621562
3,2015-01-31,41.919998,-8.75,1.458897,0.401034,9.624806
4,2015-01-31,41.919998,-8.50,1.643837,0.441838,10.604123
...,...,...,...,...,...,...
116419,2018-12-31,36.919998,-7.25,0.896764,0.025910,0.621832
116420,2018-12-31,36.919998,-7.00,0.959094,0.031726,0.761430
116421,2018-12-31,36.919998,-6.75,1.078552,0.042453,1.018877
116422,2018-12-31,36.919998,-6.50,1.389501,0.038326,0.919827


#### T2M

In [33]:
monthly_t2m_mean_df = historical_deviations[['time', 'latitude', 'longitude', 't2m']].copy()
monthly_t2m_mean_df = monthly_t2m_mean_df.rename(columns = {'t2m' : 't2m_hd'})
monthly_t2m_std_df = erA5_std_df[['time', 'latitude', 'longitude', 't2m']].copy()
monthly_t2m_std_df = monthly_t2m_std_df.rename(columns = {'t2m' : 't2m_std'})


In [34]:
monthly_t2m_mean_df.to_parquet("/Volumes/Dhruv_External_Disk/climate_data/temperature/monthly_t2m_hd_portugal_2000_2022.parquet", engine="pyarrow", compression="snappy")
monthly_t2m_std_df.to_parquet("/Volumes/Dhruv_External_Disk/climate_data/temperature/monthly_t2m_std_portugal_2000_2022.parquet", engine="pyarrow", compression="snappy")


#### TP

In [35]:
monthly_tp_mean_df = historical_deviations[['time', 'latitude', 'longitude', 'tp']].copy()
monthly_tp_mean_df = monthly_tp_mean_df.rename(columns = {'tp' : 'tp_hd'})

monthly_tp_std_df_mm_hour = erA5_std_df[['time', 'latitude', 'longitude', 'tp']].copy()
monthly_tp_std_df_mm_hour = monthly_tp_std_df_mm_hour.rename(columns = {'tp' : 'tp_std_mm_hour'})


monthly_tp_std_df = erA5_std_df[['time', 'latitude', 'longitude', 'tp_mm_day']].copy()
monthly_tp_std_df = monthly_tp_std_df.rename(columns = {'tp_mm_day' : 'tp_std'})


In [36]:
monthly_tp_mean_df.to_parquet("/Volumes/Dhruv_External_Disk/climate_data/precipitation/monthly_tp_hd_portugal_2000_2022.parquet", engine="pyarrow", compression="snappy")
monthly_tp_std_df_mm_hour.to_parquet("/Volumes/Dhruv_External_Disk/climate_data/precipitation/monthly_tp_std_mmhour_portugal_2000_2022.parquet", engine="pyarrow", compression="snappy")
monthly_tp_std_df.to_parquet("/Volumes/Dhruv_External_Disk/climate_data/precipitation/monthly_tp_std_portugal_2000_2022.parquet", engine="pyarrow", compression="snappy")